In [43]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import requests
import json
import time
import asyncio
from pymongo import MongoClient
from datetime import datetime
import csv
from google_trans_new import google_translator
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0 

In [44]:
def traducir(texto):
    contenido_p = ''
    while True:
        try:
            contenido_p = ts.translate_text(
                texto, translator='bing', to_language='es')
            break
        except Exception as e:
            print(e)
            try:
                contenido_p = ts.translate_text(
                    texto, translator='google', to_language='es')
                break
            except Exception as e:
                print(e)
                try:
                    translator = google_translator() 
                    contenido_p = translator.translate(texto, lang_tgt='es')
                except Exception as e:
                    print(e)
                    pass
                pass
            pass
    return contenido_p

In [45]:
try:
    df_list_novels = pd.read_csv('list_popular_novels_novelbin.csv', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
except:
    df_list_novels = pd.DataFrame(columns=['nombre', 'url'])

In [46]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

In [47]:
# url = 'https://novel-bin.net/sort/daily-novel-bin'
# url='https://novel-bin.net/sort/hot-novel-bin'
# url = 'https://novel-bin.net/sort/completed-novel-bin'
url='https://novel-bin.net/sort/popular-novel-bin'

In [48]:
driver = uc.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))

In [49]:
driver.get(url)
# driver.maximize_window()

In [50]:
# if df_list_novels.empty:
#     time.sleep(3)
#     list_novels = {"nombre":[],"url":[]}
#     page = 2
#     while page < 231:
#         html = driver.page_source
#         soup = bs(html, 'html.parser')
#         list_novels_page = soup.find_all('h3', class_='novel-title')
#         for a_list in list_novels_page:
#             list_novels['nombre'].append(a_list.find('a').getText())
#             list_novels['url'].append(a_list.find('a').get('href'))
#             print(a_list.find('a').getText(),a_list.find('a').get('href'))
#         time.sleep(1)
#         driver.get(f"{url}?page={page}")
#         page+= 1
#     df_list_novels = pd.DataFrame(list_novels)

In [51]:
# df_list_novels.to_csv('list_popular_novels_novelbin.csv', index=False, encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
# df_list_novels

In [52]:
client = MongoClient('mongodb://localhost:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']

In [53]:
# coleccion_app_novela = db['app_novela']
# for novela in coleccion_app_novela.find():
#     if type(novela['genero']) in (tuple, list):
#         argenero = ', '.join(novela['genero'])
#         coleccion_app_novela.update_one({'_id': novela['_id']}, {
#                                         '$set': {'genero': argenero}})
#     # break

In [54]:
def get_novel_data(driver, novel_url):
    driver.get(novel_url)
    time.sleep(1)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')

    title = soup_novel.find('h3', class_='title').getText()
    table_info = soup_novel.find('ul', class_='info info-meta').find_all('li')
    author = ''
    genre = ''
    status = ''
    for li in table_info:
        if 'Author:' in li.find('h3').getText().strip().rstrip():
            author = li.find('a').getText().strip().rstrip()
        if 'Genre:' in li.find('h3').getText().strip().rstrip():
            genre = ', '.join([x.getText().strip().rstrip() for x in li.find_all('a')])
        if 'Status:' in li.find('h3').getText().strip().rstrip():
            status = li.find('a').getText().strip().rstrip()

    img_src = soup_novel.find('div', class_='book').find('img').get('src')
    description = soup_novel.find('div', class_='desc-text').getText().strip().rstrip()

    novel_data = {
        "sitio_id": "65e7e1707963e189aa4c1256",
        "nombre": title.upper(),
        "sinopsis": description,
        "autor": author,
        "genero": genre,
        "status": 'emision' if status == 'Ongoing' else 'completo',
        "url": novel_url,
        "imagen_url": img_src,
        "created_at": datetime.now(),
        "updated_at": datetime.now()
    }

    return novel_data

In [59]:
def get_chapter_data(driver, novel_id, collection):
    driver.find_element(By.XPATH, '/html/body/div/main/div[2]/div[1]/div[4]/ul/li[2]/a').click()
    time.sleep(1)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')
    panel_chapter = soup_novel.find('div', class_='panel-body').find_all('div', class_='row')

    existing_chapters = {x['url'] for x in collection.find({'novela_id': novel_id}, {'url': 1})}

    for row in panel_chapter:
        lists_chapters = row.find_all('ul', class_='list-chapter')
        for ul in lists_chapters:
            a_list = ul.find_all('a')
            for a in a_list:
                chapter_url = str(a.get('href'))
                if chapter_url not in existing_chapters:
                    chapter_data = {
                        "novela_id": novel_id,
                        "nombre": str(a.getText()).strip().rstrip(),
                        "url": chapter_url,
                        "created_at": datetime.now(),
                        "updated_at": datetime.now()
                    }
                    collection.insert_one(chapter_data)

In [56]:
# coleccion_app_novela = db['app_novela']
# coleccion_app_capitulo = db['app_capitulo']

# for idx, novel in df_list_novels.iterrows():
#         print(idx, novel['nombre'], novel['url'])
#         result = coleccion_app_novela.find_one({'url': novel['url']})
#         if result:
#             print(f"La novela '{result['nombre']}' ya está registrada en la base de datos.")
#         else:
#             print("Novela no registrada, procediendo a obtener datos...")
#             try:
#                 novel_data = get_novel_data(driver, novel['url'])
#                 novel_id = str(coleccion_app_novela.insert_one(novel_data).inserted_id)
#                 print(f"Novela '{novel_data['nombre']}' ingresada a la Base de Datos.")
#             except Exception as e:
#                 print(f"Error al procesar la novela '{novel['url']}': {e}")
#         try:
#             get_chapter_data(driver, novel_id, coleccion_app_capitulo)
#             print(f"Todos los capítulos publicados de '{novel_data['nombre']}' han sido ingresados a la Base de Datos.")
#         except Exception as e:
#             print(f"Error al procesar la novela '{novel['url']}': {e}")

In [60]:
coleccion_app_novela = db['app_novela']
coleccion_app_capitulo = db['app_capitulo']
existing_chapters = [{"_id":str(x['_id']),"nombre":x['nombre'],"url":x['url']} for x in coleccion_app_novela.find({'sitio_id': '65e7e1707963e189aa4c1256'}, {'_id':1,'nombre':1, 'url': 1})]

for idx, novel in enumerate(existing_chapters):
        print(idx, novel['_id'], novel['nombre'], novel['url'])
        try:
            get_novel_data(driver, novel['url'])
            get_chapter_data(driver, str(novel['_id']), coleccion_app_capitulo)
            print(f"Todos los capítulos publicados de '{novel['nombre']}' han sido ingresados a la Base de Datos.")
        except Exception as e:
            print(f"Error al procesar la novela '{novel['url']}': {e}")

0 65e7ed3d9297a4c7c5aaaeef MARTIAL KING'S RETIRED LIFE https://novel-bin.net/novel-bin/martial-kings-retired-life-nov169782885
Todos los capítulos publicados de 'MARTIAL KING'S RETIRED LIFE' han sido ingresados a la Base de Datos.
1 65e874957a145405d0b24bf5 CULTIVATION CHAT GROUP https://novel-bin.net/novel-bin/cultivation-chat-group-nov1827941102
Todos los capítulos publicados de 'CULTIVATION CHAT GROUP' han sido ingresados a la Base de Datos.
2 65efbb2ceb6e829adc4a104a EMPEROR'S DOMINATION https://novel-bin.net/novel-bin/emperors-domination-nov1762445829
Todos los capítulos publicados de 'EMPEROR'S DOMINATION' han sido ingresados a la Base de Datos.
3 65efbb58eb6e829adc4a104c BECOMING IMMORTAL THROUGH GETTING MARRIED https://novel-bin.net/novel-bin/becoming-immortal-through-getting-married-nov-1998792937
Todos los capítulos publicados de 'BECOMING IMMORTAL THROUGH GETTING MARRIED' han sido ingresados a la Base de Datos.
4 65f1af5d4285ef9618f6b28f AS AN IMMORTAL, I ONLY LEARN FORBIDDE

In [ ]:
# coleccion_app_capitulo = db['app_capitulo']
# capitulos_novelas_base = coleccion_app_capitulo.find()

In [ ]:
# for capitulo in capitulos_novelas_base:
#     if detect(capitulo['nombre']) == 'en':
#         print(capitulo['_id'],detect(capitulo['nombre']),capitulo['nombre'])
#         coleccion_app_capitulo.update_one({'_id': capitulo['_id']}, {
#             '$set': {'nombre': traducir(capitulo['nombre'])}
#         })
#     # break

In [61]:
driver.quit()